<a href="https://colab.research.google.com/github/ereny123/Multitasking-MThesis-2024/blob/main/Pytorch_NET_Multitasking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#create a model class
class Model(nn.Module):
  #Input (Tasks) random array of 9 units
  def __init__(self, in_tasks=9, h1= 100, output=9):
    super().__init__()
    self.fc1=nn.Linear(in_tasks, h1)
    self.fc2=nn.Linear(h1, output)

  def forward(self, x):
    x= F.leaky_relu(self.fc1(x))
    x=self.out(x)

    return x




In [ ]:
#Pick a manual seed for randomization for the input values
torch.manual_seed(1729)
#tasks= torch.rand(1,9)
#tasks


#create an instance for Model
model = Model()
